In [1]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
import os
import pandas as pd
import numpy as np 
import math
import json
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

### 封装函数

In [2]:
os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\3-regression-model")

In [173]:
df0 = pd.read_csv('nlsy/nlsy_stage_2.csv').drop('Unnamed: 0', axis=1).dropna()

In [175]:
df0.columns

Index(['person_id', 'single_couple', 'couple_single', 'single_single',
       'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',
       'education_level', 'first_job_income_quantile', 'noncentral_central',
       'central_noncentral', 'num_of_event', 'event_weighted_score',
       'event_entropy_score', 'prev_stage', 'gender', 'birth_year', 'race',
       'dv_stage'],
      dtype='object')

In [ ]:
nlsw_cluster_n, nlsw_cluster_n = 7,9 # cluster 1-7, cluster 0-8
nlsm_cluster_n, nlsm_cluster_n = 3,5
nlsy_cluster_n, nlsy_cluster_n = 8,9
psid_cluster_n, psid_cluster_n = 10,10

edu_rename_map = {
    'high-school-noncompletion':'edu0',
    'high-school':'edu1',
    'college':'edu2'
}
# 预处理的map
firstjob_rename_map = dict(zip(range(1,6),[f'firstjob{i}' for i in range(5)]))
prevcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(nlsw_cluster_n)], [f'prevcluster{i}' for i in range(nlsw_cluster_n)]))
dvcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(nlsw_cluster_n)], [f'dvcluster{i}' for i in range(nlsw_cluster_n)]))
# 预处理
df0['education_level']=df0['education_level'].apply(lambda x: edu_rename_map[x])
df0['first_job_income_quantile']=df0['first_job_income_quantile'].apply(lambda x: firstjob_rename_map[x])
df0['prev_stage']=df0['prev_stage'].apply(lambda x: prevcluster_rename_map[x])
df0['dv_stage']=df0['dv_stage'].apply(lambda x: dvcluster_rename_map[x])
# 索引（得到结果之后）
edu_map = dict(zip(list(edu_rename_map.values()),list(edu_rename_map.keys())))
firstjob_map = dict(zip(list(firstjob_rename_map.values()),list(firstjob_rename_map.keys())))
prevcluster_map = dict(zip(list(prevcluster_rename_map.values()),list(prevcluster_rename_map.keys())))
dvcluster_map = dict(zip(list(dvcluster_rename_map.values()),list(dvcluster_rename_map.keys())))

# 有用的参数
num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
cat_control_var_list = [ 'gender', 'race']
bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration'] # nlsy migration
dependent_var = ['dv_stage']
control_var_list = ['gender','race','birth_year']

dvcluster_list = [f'dvcluster{i}' for i in range(nlsw_cluster_n)]
#nlsw_cat_dict.update(cv_control_dict)
nlsw_cats_dict = {
    'education_level': [ 'edu1', 'edu2' ], # nlsw 1,2
    'first_job_income_quantile': [f'firstjob{i}' for i in range(5)],
    'prev_stage': [f'prevcluster{i}' for i in range(nlsw_cluster_n)],
    'dv_stage': [f'dvcluster{i}' for i in range(nlsw_cluster_n)],
}   

control_dict = {    
    'gender': ['male','female'],
    'race': ['white','black','other']}

refs_dict = {}
for var in list(nlsw_cats_dict.keys()):
    cats=[]
    cat = nlsw_cats_dict[var]
    for i in range(len(cat)):
        cats.append(cat)
        cat = cat[1:]+[cat[0]]
    refs_dict[var]=cats
#'event_weighted_score','event_entropy_score',

In [129]:
edu_map = dict(zip(list(edu_rename_map.values()),list(edu_rename_map.keys())))
firstjob_map = dict(zip(list(firstjob_rename_map.values()),list(firstjob_rename_map.keys())))
prevcluster_map = dict(zip(list(prevcluster_rename_map.values()),list(prevcluster_rename_map.keys())))
dvcluster_map = dict(zip(list(dvcluster_rename_map.values()),list(dvcluster_rename_map.keys())))

In [130]:
df0['education_level']=df0['education_level'].apply(lambda x: edu_rename_map[x])
df0['first_job_income_quantile']=df0['first_job_income_quantile'].apply(lambda x: firstjob_rename_map[x])
df0['prev_stage']=df0['prev_stage'].apply(lambda x: prevcluster_rename_map[x])
df0['dv_stage']=df0['dv_stage'].apply(lambda x: dvcluster_rename_map[x])

In [3]:
num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
cat_control_var_list = [ 'gender', 'race']
bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration'] # nlsy migration
dependent_var = ['dv_stage']
control_var_list = ['gender','race','birth_year']

dvcluster_list = [f'dvcluster{i}' for i in range(nlsw_cluster_n)]
#nlsw_cat_dict.update(cv_control_dict)
nlsw_cats_dict = {
    'education_level': [ 'edu1', 'edu2' ], # nlsw 1,2
    'first_job_income_quantile': [f'firstjob{i}' for i in range(5)],
    'prev_stage': [f'prevcluster{i}' for i in range(nlsw_cluster_n)],
    'dv_stage': [f'dvcluster{i}' for i in range(nlsw_cluster_n)],
}   

control_dict = {    
    'gender': ['male','female'],
    'race': ['white','black','other']}

refs_dict = {}
for var in list(nlsw_cats_dict.keys()):
    cats=[]
    cat = nlsw_cats_dict[var]
    for i in range(len(cat)):
        cats.append(cat)
        cat = cat[1:]+[cat[0]]
    refs_dict[var]=cats
#'event_weighted_score','event_entropy_score',


NameError: name 'nlsw_cluster_n' is not defined

## 格式

In [ ]:
# coef_p_value.json
# Loop 1: for each model, 
# Loop 2: for each Reference Group in the independent variables, 
# Loop 3: for each Reference Group in the dependent variable, 
# pls output below dict:
{
   "dvcluster0": {
      "edu0_firstjob0_prevcluster0_dvcluster_0": {
         "single_couple": {"p-value": xx, "coef": xx, "e_coef": xx},  # 下面用"{...}"代替这个具体内容
         "couple_single": {...}, # 同上
         "single_single": {...}, # 同上
         "childbearing_no_yes": {...}, # 同上
         "childbearing_yes_no": {...}, # 同上
         "has_migration": {...}, # 同上
          # "central_nocentral" 这两个只出现在NLSY dataset中
          # "nocentral_central"
         "education_level": {
             "edu0_edu1": {...}, # category0/1/2: 是具体degree名称。下划线前面是reference group, 后面是target group
             "edu0_edu2": {...}, 
         }, 
         "first_job_income_quantile": {......}, # 同 education_level

         "num_of_event": {...}, # 同上
         "event_weighted_score": {...}, # 同上
         "event_entropy_score": {...}, # 同上

         "prev_stage": {......}, # 同 education_level
         "gender" # control variable不输出
         "birth_year" # control variable不输出
         "race" # control variable不输出
         "dv_stage" # DV不输出
      },
      "edu0_firstjob0_prevcluster0_dvcluster_1": {...}, 
      "edu0_firstjob0_prevcluster0_dvcluster_2": {...}, 
      "edu0_firstjob0_prevcluster0_dvcluster_3": {...}, 
      # ...
      "edu0_firstjob0_prevcluster1_dvcluster_0": {...}, 
      "edu0_firstjob0_prevcluster1_dvcluster_1": {...}, 
      "edu0_firstjob0_prevcluster1_dvcluster_2": {...}, 
      # ...
      "edu0_firstjob0_prevcluster2_dvcluster_0": {...}, 
      "edu0_firstjob0_prevcluster2_dvcluster_1": {...}, 
      "edu0_firstjob0_prevcluster2_dvcluster_2": {...}, 
      # ...
   },
   "cluster1": {...}, 
   "cluster2": {...}, 
   "cluster3": {...}, 
   ...
   "cluster9": {...}, 
}

In [ ]:
# lrt.json
# pick one reference group in each categorical IVs and DV, to get the following results
{
   "single_couple": {"lrt": xx, "degrees_of_freedom": xx},  # 下面用"{...}"代替这个具体内容
   "couple_single": {...}, # 同上
   "single_single": {...}, # 同上
   "childbearing_no_yes": {...}, # 同上
   "childbearing_yes_no": {...}, # 同上
   "has_migration": {...}, # 同上
   # "central_nocentral" 这两个只出现在NLSY dataset中
   # "nocentral_central"
   "education_level": {...}, # 同上
   "first_job_income_quantile": {...}, # 同上

   "num_of_event": {...}, # 同上
   "event_weighted_score": {...}, # 同上
   "event_entropy_score": {...}, # 同上

   "prev_stage": {...}, # 同上
}

## 实验1

### coef_p_value.json

In [9]:
nlsw_stage_n, nlsw_cluster_n = 7,9 # stage 1-7, cluster 0-8
nlsm_stage_n, nlsm_cluster_n = 3,5
nlsy_stage_n, nlsy_cluster_n = 8,9
psid_stage_n, psid_cluster_n = 10,10

In [ ]:
# 有用的参数
num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
cat_control_var_list = [ 'gender', 'race']
bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
dependent_var = ['dv_stage']
control_var_list = ['gender','race','birth_year']

dvcluster_list = [f'dvcluster{i}' for i in range(nlsy_cluster_n)]
#nlsy_cat_dict.update(cv_control_dict)
nlsy_cats_dict = {
    'education_level': ['edu0' ,'edu1', 'edu2' ], # nlsw,nlsm 1,2; psid,nlsy 0,1,2
    'first_job_income_quantile': [f'firstjob{i}' for i in range(5)],
    'prev_stage': [f'prevcluster{i}' for i in range(nlsy_cluster_n)],
    'dv_stage': [f'dvcluster{i}' for i in range(nlsy_cluster_n)],
}   

control_dict = {    
    'gender': ['male','female'],
    'race': ['white','black','other']}

refs_dict = {}
for var in list(nlsy_cats_dict.keys()):
    cats=[]
    cat = nlsy_cats_dict[var]
    for i in range(len(cat)):
        cats.append(cat)
        cat = cat[1:]+[cat[0]]
    refs_dict[var]=cats
#'event_weighted_score','event_entropy_score',
    


coef_p_value_dict = dict(zip([f'stage{i+1}' for i in range(nlsy_stage_n)],[dict(zip(dvcluster_list,[0 for j in range(nlsy_cluster_n)])) for k in range(nlsy_stage_n)]))


for stage_k in range(1,nlsy_stage_n):
    df0 = pd.read_csv('nlsy/nlsy_stage_'+str(stage_k+1)+'.csv').drop('Unnamed: 0', axis=1).dropna()
    # 预处理的map
    edu_rename_map = {
        'high-school-noncompletion':'edu0',
        'high-school':'edu1',
        'college':'edu2'
    }
    firstjob_rename_map = dict(zip(range(1,6),[f'firstjob{i}' for i in range(5)]))
    prevcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(nlsy_cluster_n)], [f'prevcluster{i}' for i in range(nlsy_cluster_n)]))
    dvcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(nlsy_cluster_n)], [f'dvcluster{i}' for i in range(nlsy_cluster_n)]))
    # 预处理
    df0['education_level']=df0['education_level'].apply(lambda x: edu_rename_map[x])
    df0['first_job_income_quantile']=df0['first_job_income_quantile'].apply(lambda x: firstjob_rename_map[x])
    df0['prev_stage']=df0['prev_stage'].apply(lambda x: prevcluster_rename_map[x])
    df0['dv_stage']=df0['dv_stage'].apply(lambda x: dvcluster_rename_map[x])
    # 索引（得到结果之后）
    edu_map = dict(zip(list(edu_rename_map.values()),list(edu_rename_map.keys())))
    firstjob_map = dict(zip(list(firstjob_rename_map.values()),list(firstjob_rename_map.keys())))
    prevcluster_map = dict(zip(list(prevcluster_rename_map.values()),list(prevcluster_rename_map.keys())))
    dvcluster_map = dict(zip(list(dvcluster_rename_map.values()),list(dvcluster_rename_map.keys())))

    
    
    input_X = df0.iloc[:,:-1]
    
    for ref_dv in dvcluster_list:
        dv_rest_list = [f'dvcluster{i}' for i in range(nlsy_cluster_n)]
        dv_rest_list.remove(ref_dv) 
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})
        coef_p_value_dict['stage'+str(stage_k+1)][ref_dv] = {}
        
        for ref_pv in refs_dict['prev_stage']:
            input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=ref_pv)
            for ref_firstjob in refs_dict['first_job_income_quantile']:
                input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=ref_firstjob)
                for ref_edu in refs_dict['education_level']:
                    input_X['education_level']=pd.Categorical(input_X['education_level'], categories=ref_edu)
                    ref_groups_iv = [ref_edu, ref_firstjob, ref_pv] # 对应三个简写
                    
                    cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
                    train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
                    sc_X = StandardScaler()
                    train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

                    train_y = input_y
                    mlogit_mod = sm.MNLogit(train_y, train_X_sd)
                    mlogit_res = mlogit_mod.fit(method='bfgs')
                    parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
                    p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues


                    # 输出到指定格式
                    iv_ref_string = '_'+ref_edu[0]+'_'+ref_firstjob[0]+'_'+ref_pv[0]
                    ## 创建储存各dvcluster的dict
                    new_dict = dict(zip([dv_rest_list[i]+iv_ref_string for i in range(parameters.shape[1])], [0 for i in range(parameters.shape[1])]))
                    
                    for i in range(parameters.shape[1]):
                        l0=[p_values[i], parameters[i], parameters[i].apply(lambda x: math.exp(x))]
                        df = pd.concat(l0, axis=1)
                        df.columns = ['p-value', 'coef', 'e_coef'] 
                        label = dv_rest_list[i]+iv_ref_string
                        new_dict[label] = df.loc[num_var_list+bool_var_list].to_dict('index')

                        for idx, ch_cat in enumerate(cat_change_var_list):
                            names = [k for k in list(df.index) if k.startswith(ch_cat)]
                            df1 = df.loc[names]
                            df1.index = [k.replace(ch_cat,ref_groups_iv[idx][0]) for k in names]
                            new_dict[label][ch_cat] = df1.to_dict('index')           
                    # 一组新的iv ref groups：更新到coef_p_value_dict[stage][ref_dv]中               
                    coef_p_value_dict['stage'+str(stage_k+1)][ref_dv].update(new_dict) 
            
# cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
# train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
# sc_X = StandardScaler()
# train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

# train_y = input_y
# mlogit_mod = sm.MNLogit(train_y, train_X_sd)
# mlogit_res = mlogit_mod.fit(method='bfgs')

# y0 = mlogit_res.predict()
# ll =pd.DataFrame(train_y).dropna()
# ll.index = range(ll.shape[0])
# def findmax(x):
#     mxx=x.max()
#     y = list(x)
#     return 's_cluster_'+str(y.index(mxx))

# k = pd.concat([ll,pd.DataFrame(y0).apply(lambda x: findmax(x),axis=1)],axis=1,ignore_index=True).apply(lambda x: x[0]==x[1],axis=1).value_counts(normalize=True)
# print(k[True])

In [15]:
stage_k

1

### psid stage9,10

In [ ]:
# 有用的参数
num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
cat_control_var_list = [ 'gender', 'race']
bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration'] # nlsy migration
dependent_var = ['dv_stage']
control_var_list = ['gender','race','birth_year']

dvcluster_list = [f'dvcluster{i}' for i in range(psid_cluster_n)]
#psid_cat_dict.update(cv_control_dict)
psid_cats_dict = {
    'education_level': ['edu0' ,'edu1', 'edu2' ], # nlsw,nlsm 1,2; nlsy,psid 0,1,2
    'first_job_income_quantile': [f'firstjob{i}' for i in range(5)],
    'prev_stage': [f'prevcluster{i}' for i in range(psid_cluster_n)],
    'dv_stage': [f'dvcluster{i}' for i in range(psid_cluster_n)],
}   

control_dict = {    
    'gender': ['male','female'],
    'race': ['white','black','other']}

refs_dict = {}
for var in list(psid_cats_dict.keys()):
    cats=[]
    cat = psid_cats_dict[var]
    for i in range(len(cat)):
        cats.append(cat)
        cat = cat[1:]+[cat[0]]
    refs_dict[var]=cats
#'event_weighted_score','event_entropy_score',


for stage_k in range(9,psid_stage_n):
    df0 = pd.read_csv('psid/psid_stage_'+str(stage_k+1)+'.csv').drop('Unnamed: 0', axis=1).dropna()
    # 预处理的map
    edu_rename_map = {
        'high-school-noncompletion':'edu0',
        'high-school':'edu1',
        'college':'edu2'
    }
    firstjob_rename_map = dict(zip(range(1,6),[f'firstjob{i}' for i in range(5)]))
    prevcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(psid_cluster_n)], [f'prevcluster{i}' for i in range(psid_cluster_n)]))
    dvcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(psid_cluster_n)], [f'dvcluster{i}' for i in range(psid_cluster_n)]))
    # 预处理
    df0['education_level']=df0['education_level'].apply(lambda x: edu_rename_map[x])
    df0['first_job_income_quantile']=df0['first_job_income_quantile'].apply(lambda x: firstjob_rename_map[x])
    df0['prev_stage']=df0['prev_stage'].apply(lambda x: prevcluster_rename_map[x])
    df0['dv_stage']=df0['dv_stage'].apply(lambda x: dvcluster_rename_map[x])
    # 索引（得到结果之后）
    edu_map = dict(zip(list(edu_rename_map.values()),list(edu_rename_map.keys())))
    firstjob_map = dict(zip(list(firstjob_rename_map.values()),list(firstjob_rename_map.keys())))
    prevcluster_map = dict(zip(list(prevcluster_rename_map.values()),list(prevcluster_rename_map.keys())))
    dvcluster_map = dict(zip(list(dvcluster_rename_map.values()),list(dvcluster_rename_map.keys())))

    
    
    input_X = df0.iloc[:,:-1]
    
    for ref_dv in dvcluster_list:
        dv_rest_list = [f'dvcluster{i}' for i in range(psid_cluster_n)]
        dv_rest_list.remove(ref_dv) 
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})
        coef_p_value_dict['stage'+str(stage_k+1)][ref_dv] = {}
        
        for ref_pv in refs_dict['prev_stage']:
            input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=ref_pv)
            for ref_firstjob in refs_dict['first_job_income_quantile']:
                input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=ref_firstjob)
                for ref_edu in refs_dict['education_level']:
                    input_X['education_level']=pd.Categorical(input_X['education_level'], categories=ref_edu)
                    ref_groups_iv = [ref_edu, ref_firstjob, ref_pv] # 对应三个简写
                    
                    cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
                    train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
                    sc_X = StandardScaler()
                    train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

                    train_y = input_y
                    
                    if ref_firstjob[0]=='firstjob2':
                        m ='cg'
                    else:
                        m='bfgs'
                    mlogit_mod = sm.MNLogit(train_y, train_X_sd)
                    mlogit_res = mlogit_mod.fit(method=m)
                    parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
                    p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues


                    # 输出到指定格式
                    iv_ref_string = '_'+ref_edu[0]+'_'+ref_firstjob[0]+'_'+ref_pv[0]
                    ## 创建储存各dvcluster的dict
                    new_dict = dict(zip([dv_rest_list[i]+iv_ref_string for i in range(parameters.shape[1])], [0 for i in range(parameters.shape[1])]))
                    
                    for i in range(parameters.shape[1]):
                        l0=[p_values[i], parameters[i], parameters[i].apply(lambda x: math.exp(x))]
                        df = pd.concat(l0, axis=1)
                        df.columns = ['p-value', 'coef', 'e_coef'] 
                        label = dv_rest_list[i]+iv_ref_string
                        new_dict[label] = df.loc[num_var_list+bool_var_list].to_dict('index')

                        for idx, ch_cat in enumerate(cat_change_var_list):
                            names = [k for k in list(df.index) if k.startswith(ch_cat)]
                            df1 = df.loc[names]
                            df1.index = [k.replace(ch_cat,ref_groups_iv[idx][0]) for k in names]
                            new_dict[label][ch_cat] = df1.to_dict('index')           
                    # 一组新的iv ref groups：更新到coef_p_value_dict[stage][ref_dv]中               
                    coef_p_value_dict['stage'+str(stage_k+1)][ref_dv].update(new_dict) 

In [ ]:
ref_edu, ref_firstjob, ref_pv

In [178]:
with open("nlsy/nlsy-coef_p_value.json", 'w') as f:
    json.dump(coef_p_value_dict,f, indent = 2, sort_keys=True)

### lrt.json

In [7]:
df0 = pd.read_csv('psid/psid_stage_2.csv').drop('Unnamed: 0', axis=1).dropna()
input_X = df0.iloc[:,:-1]
drop_var_list = [k for k in input_X.columns if k not in control_var_list+['person_id']]
LR_dict = dict(zip(['full_model']+drop_var_list, [{'loglikelihood':dict(zip(['stage'+str(i+1) for i in range(1,psid_stage_n)], [0 for i in range(1,psid_stage_n)])),'degree_of_freedom':0}]+[{'LR': dict(zip(['stage'+str(i+1) for i in range(1,psid_stage_n)], [0 for i in range(1,psid_stage_n)])), 'degree_of_freedom': 0} for i in range(len(drop_var_list))]))

In [8]:
# 有用的参数
num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
cat_control_var_list = [ 'gender', 'race']
bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no',]# 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
dependent_var = ['dv_stage']
control_var_list = ['gender','race','birth_year']

dvcluster_list = [f'dvcluster{i}' for i in range(psid_cluster_n)]
#psid_cat_dict.update(cv_control_dict)

control_dict = {    
    'gender': ['male','female'],
    'race': ['white','black','other']}



for i in range(1,psid_stage_n):
    df0 = pd.read_csv('psid/psid_stage_'+str(i+1)+'.csv').drop('Unnamed: 0', axis=1).dropna()
    # 预处理的map
    edu_rename_map = {
        'high-school-noncompletion':'edu0',
        'high-school':'edu1',
        'college':'edu2'
    }
    firstjob_rename_map = dict(zip(range(1,6),[f'firstjob{i}' for i in range(5)]))
    prevcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(psid_cluster_n)], [f'prevcluster{i}' for i in range(psid_cluster_n)]))
    dvcluster_rename_map = dict(zip([f's_cluster_{i}' for i in range(psid_cluster_n)], [f'dvcluster{i}' for i in range(psid_cluster_n)]))
    # 预处理
    df0['education_level']=df0['education_level'].apply(lambda x: edu_rename_map[x])
    df0['first_job_income_quantile']=df0['first_job_income_quantile'].apply(lambda x: firstjob_rename_map[x])
    df0['prev_stage']=df0['prev_stage'].apply(lambda x: prevcluster_rename_map[x])
    df0['dv_stage']=df0['dv_stage'].apply(lambda x: dvcluster_rename_map[x])
    
    input_X = df0.iloc[:,:-1]
    input_y = df0['dv_stage']
    
    psid_cats_dict = {
    'education_level': ['edu0','edu1', 'edu2' ], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [f'firstjob{i}' for i in range(5)],
    'prev_stage': [f'prevcluster{i}' for i in range(psid_cluster_n)],
    'dv_stage': [f'dvcluster{i}' for i in range(psid_cluster_n)],
    }  
    
    # 赋予categorical任一顺序
    input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=psid_cats_dict['prev_stage'])
    input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=psid_cats_dict['first_job_income_quantile'])
    input_X['education_level']=pd.Categorical(input_X['education_level'], categories=psid_cats_dict['education_level'])

    

    cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
    train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
    sc_X = StandardScaler()
    train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

    train_y = input_y
    mlogit_mod = sm.MNLogit(train_y, train_X_sd)
    mlogit_res = mlogit_mod.fit(method='bfgs')
    full_llf  = mlogit_res.llf
    full_df = mlogit_res.df_model
    
    LR_dict['full_model']['degree_of_freedom']=full_df
    LR_dict['full_model']['loglikelihood']['stage'+str(i+1)] = full_llf
    
    drop_var_list = [k for k in input_X.columns if k not in control_var_list+['person_id']]
    
    for var in drop_var_list:
        cat_var_list1=[k for k in cat_var_list if k!=var]
        num_var_list1=[k for k in num_var_list if k!=var]
        bool_var_list1=[k for k in bool_var_list if k!=var]

        cat_dummies1 = pd.get_dummies(input_X[cat_var_list1], drop_first=True)
        train_X_drop = pd.concat([input_X[num_var_list1+bool_var_list1], cat_dummies1], axis=1)
        sc_X = StandardScaler()
        train_X_drop_sd = pd.DataFrame(sc_X.fit_transform(train_X_drop), index=train_X_drop.index, columns=train_X_drop.columns)

        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_drop_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        var_llf  = mlogit_res.llf
        var_df = mlogit_res.df_model
        
        LR = 2*(full_llf-var_llf)
        df = full_df-var_df

        LR_dict[var]['degree_of_freedom'] = df
        LR_dict[var]['LR']['stage'+str(i+1)] = LR
        

         Current function value: 1.496824
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.497211
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.498117
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.497370
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.497469
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.497099
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.496824
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.512864
         Iterations:

KeyboardInterrupt: 

In [ ]:
LR_dict

In [ ]:
mlogit_res.summary()

In [60]:
with open("psid/psid-lrt.json", 'w') as f:
    json.dump(LR_dict,f, indent = 2, sort_keys=True)

## 实验2：flow

### group

In [3]:
nlsw_stage_n, nlsw_cluster_n = 7,9 # stage 1-7, cluster 0-8
nlsm_stage_n, nlsm_cluster_n = 3,5
nlsy_stage_n, nlsy_cluster_n = 8,9
psid_stage_n, psid_cluster_n = 10,10

dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
    'cats_dict':{ # 之后删去，用if来赋值
        'nlsw': {
        'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
        'first_job_income_quantile': [i+1 for i in range(5)],
        'prev_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
        },
        'nlsm': {
        'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
        'first_job_income_quantile': [i+1 for i in range(5)],
        'prev_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
        },
        'nlsy': {
        'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
        'first_job_income_quantile': [i+1 for i in range(5)],
        'prev_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
        },
        'nlsm': {
        'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
        'first_job_income_quantile': [i+1 for i in range(5)],
        'prev_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
        },
    },
}

In [8]:
pd.read_csv('nlsw/nlsw_stage_2.csv').drop('Unnamed: 0', axis=1).dropna()['prev_stage'].value_counts()

s_cluster_6    256
s_cluster_8    232
s_cluster_0    227
s_cluster_2    220
s_cluster_7    208
s_cluster_1    174
s_cluster_5    162
s_cluster_3    153
s_cluster_4    127
Name: prev_stage, dtype: int64

In [7]:
ref_choice_list = ['college', 2, 's_cluster_5', 's_cluster_3']
choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage', 'dv_stage']
nlsw_cats_dict = {
'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
'first_job_income_quantile': [i+1 for i in range(5)],
'prev_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
'dv_stage': [f's_cluster_{i}' for i in range(nlsw_cluster_n)],
}  

for idx, choice in enumerate(ref_choice_list):
    var = choice_order_list[idx]
    l0 = nlsw_cats_dict[var]
    l0.remove(choice)
    nlsw_cats_dict[var] = [choice]+l0

In [14]:
cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
cats_dict['nlsw']

{'education_level': ['high-school', 'college'],
 'first_job_income_quantile': [1, 2, 3, 4, 5],
 'prev_stage': ['s_cluster_0',
  's_cluster_1',
  's_cluster_2',
  's_cluster_3',
  's_cluster_4',
  's_cluster_5',
  's_cluster_6',
  's_cluster_7',
  's_cluster_8']}

In [4]:
# 获取所有模型的函数，储存在models_dict内
def get_pred_group(iv_refs, dv_ref, dataset_name): #iv_refs = ['college', 3, 's_cluster_5']; dv_ref = 's_cluster_3'
    # 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
    cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    models_dict = dict(zip([f'stage{i+1}' for i in range(1,dataset_info['stage_n'][dataset_name])],[0 for i in range(1,dataset_info['stage_n'][dataset_name])]))
    
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        ref_choice_list = iv_refs
        ref_dv = dv_ref
        choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]

        # 设定iv的ref
        for idx, choice in enumerate(ref_choice_list):
            var = choice_order_list[idx]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})

        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        # 储存模型
        models_dict['stage'+str(i+1)] = mlogit_res
    #获取预测
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        if i==1:
            df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_2.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
            input_X = df0.iloc[:,:-1]
            input_y = df0['dv_stage']
            pred_cluster_info = df0[['prev_stage']].rename(columns={'prev_stage':'real_stage1'})
            
            for var in list(cat_dict.keys()):
                input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
            cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
            train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
            sc_X = StandardScaler()
            train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
            pred_cluster_info['pred_stage2'] = pd.Series(np.asarray(models_dict['stage2'].predict(train_X_sd)).argmax(1), index=train_X.index).apply(lambda x: 's_cluster_'+str(x))
            pred_cluster_info['real_stage2'] = input_y
        else:
            df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1)
            df0['prev_stage']=pred_cluster_info['pred_stage'+str(i)]
            df0 = df0.dropna()
            input_X = df0.iloc[:,:-1]
            input_y = df0['dv_stage']
            
            for var in list(cat_dict.keys()):
                input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
            cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
            train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
            sc_X = StandardScaler()
            train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
            pred_cluster_info['pred_stage'+str(i+1)] = pd.Series(np.asarray(models_dict['stage'+str(i+1)].predict(train_X_sd)).argmax(1), index=train_X.index).apply(lambda x: 's_cluster_'+str(x))
            pred_cluster_info['real_stage'+str(i+1)] = input_y 
    
    return pred_cluster_info

#### 概率树（end剪枝版）

In [27]:
# 获取所有模型的函数，储存在models_dict内
def get_pred_group(dataset_name, grouping_criteria, group_name, ref_groups): #iv_refs = ['college', 3, 's_cluster_5']; dv_ref = 's_cluster_3'
    os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\3-regression-model")
    # 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
    cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    models_dict = dict(zip([f'stage{i+1}' for i in range(1,dataset_info['stage_n'][dataset_name])],[0 for i in range(1,dataset_info['stage_n'][dataset_name])]))
    
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        # 设定iv的ref   
        choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]
        for var in choice_order_list:
            choice = ref_groups[var]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        ref_dv = ref_groups['dv_stage']
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})

        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        # 储存模型
        models_dict['stage'+str(i+1)] = mlogit_res
    #获取预测
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_2.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        # input_X = df0.iloc[:,:-1]
        # input_y = df0['dv_stage']
        # 设置存储结果的df
        pred_cluster_info = df0[['prev_stage']].rename(columns={'prev_stage':'stage1'})
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1)
        df0['prev_stage']=pred_cluster_info['stage'+str(i)]
        df0=df0.dropna()

        
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        for var in list(cat_dict.keys()):
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
            
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        pred_cluster_info['stage'+str(i+1)] = pd.Series(np.asarray(models_dict['stage'+str(i+1)].predict(train_X_sd)).argmax(1), index=train_X.index).apply(lambda x: 's_cluster_'+str(x))
        
    pred_seq_dict = [{"person_id": idx, "predicted_stage_seq":[]} for idx in list(pred_cluster_info.index)]
    
    for person in pred_seq_dict:
        pid = person["person_id"]
        df = pred_cluster_info.loc[pid].dropna()
        person["predicted_stage_seq"]=[{"stage_name": idx, "career_cluster": df[idx] } for idx in df.index]
        
    
    
    return pred_seq_dict

In [28]:
pred_dict = get_pred_group( dataset_name='nlsy',grouping_criteria="gender", group_name= "male", ref_groups={
      "education_level": "high-school", 
      "first_job_income_quantile": 1, 
      "prev_stage": "s_cluster_1", 
      "dv_stage": "s_cluster_1", 
    },)

         Current function value: 1.758949
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.559027
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.441583
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.296271
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.278168
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.222817
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.163091
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


In [29]:
pred_dict

[{'person_id': 2,
  'predicted_stage_seq': [{'stage_name': 'stage1',
    'career_cluster': 's_cluster_1'},
   {'stage_name': 'stage2', 'career_cluster': 's_cluster_0'},
   {'stage_name': 'stage3', 'career_cluster': 's_cluster_3'},
   {'stage_name': 'stage4', 'career_cluster': 's_cluster_3'},
   {'stage_name': 'stage5', 'career_cluster': 's_cluster_3'},
   {'stage_name': 'stage6', 'career_cluster': 's_cluster_3'},
   {'stage_name': 'stage7', 'career_cluster': 's_cluster_3'},
   {'stage_name': 'stage8', 'career_cluster': 's_cluster_3'}]},
 {'person_id': 6,
  'predicted_stage_seq': [{'stage_name': 'stage1',
    'career_cluster': 's_cluster_0'},
   {'stage_name': 'stage2', 'career_cluster': 's_cluster_5'},
   {'stage_name': 'stage3', 'career_cluster': 's_cluster_5'},
   {'stage_name': 'stage4', 'career_cluster': 's_cluster_5'},
   {'stage_name': 'stage5', 'career_cluster': 's_cluster_5'},
   {'stage_name': 'stage6', 'career_cluster': 's_cluster_5'},
   {'stage_name': 'stage7', 'career_clus

In [24]:
for i in pred_dict.loc[7].dropna().index:
    print(i, pred_dict.loc[7].dropna()[i])


stage1 s_cluster_3
stage2 s_cluster_0
stage3 s_cluster_3
stage4 s_cluster_3
stage5 s_cluster_3
stage6 s_cluster_3


##### 存档（计算概率）

In [ ]:
# 获取所有模型的函数，储存在models_dict内
def get_pred_group(dataset_name, grouping_criteria, group_name, ref_groups): #iv_refs = ['college', 3, 's_cluster_5']; dv_ref = 's_cluster_3'
    os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\3-regression-model")
    # 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
    cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    models_dict = dict(zip([f'stage{i+1}' for i in range(1,dataset_info['stage_n'][dataset_name])],[0 for i in range(1,dataset_info['stage_n'][dataset_name])]))
    
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        # 设定iv的ref   
        choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]
        for var in choice_order_list:
            choice = ref_groups[var]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        ref_dv = ref_groups['dv_stage']
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})

        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        # 储存模型
        models_dict['stage'+str(i+1)] = mlogit_res
    #获取预测
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_2.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        # input_X = df0.iloc[:,:-1]
        # input_y = df0['dv_stage']
        # 设置存储结果的df
        pred_cluster_info = df0[['prev_stage']].rename(columns={'prev_stage':'stage1'})
        pred_cluster_info['cluster_seq'] = pred_cluster_info['stage1']
        pred_cluster_info['prob'] = 0
    for i in range(1,stage_k):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1)
        df0['prev_stage']=pred_cluster_info['stage'+str(i)]
        df0=df0.dropna()

        
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        for var in list(cat_dict.keys()):
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
            
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        pred_cluster_info['stage'+str(i+1)] = pd.Series(np.asarray(models_dict['stage'+str(i+1)].predict(train_X_sd)).argmax(1), index=train_X.index).apply(lambda x: 's_cluster_'+str(x))
        print(pred_cluster_info[pred_cluster_info['stage'+str(i+1)].isna()])

        stage_now = 'stage'+str(i+1)
        pred_cluster_info.loc[~pred_cluster_info[stage_now].isna(),'cluster_seq']=pred_cluster_info.dropna().apply(lambda x: x['cluster_seq']+'-'+x[stage_now] , axis=1)    
        dict0 = pred_cluster_info.dropna()['cluster_seq'].value_counts(normalize=True).to_dict()
        pred_cluster_info.loc[~pred_cluster_info[stage_now].isna(),'prob']=pred_cluster_info.dropna().apply(lambda x: dict0[x['cluster_seq']], axis=1)
        
        #pred_cluster_info.loc[pred_cluster_info[stage_now]==np.nan]['prob']=1 #df.loc[]=xx并不改变df，必须加上一个指定的位置df.loc[condition, 'var_name']
        
    
    return pred_cluster_info

In [27]:
pred_dict = get_pred_group(iv_refs = ['college', 3, 's_cluster_5'], dv_ref = 's_cluster_3', dataset_name='nlsy', threh=0.001, stage_k=5)

         Current function value: 1.756854
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.564915
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.442346
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.298118
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.285146
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.233840
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.167726
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
Empty DataFrame
Columns: [stage1, cluster_seq, prob, stage2]
I

In [35]:
pred_dict['cluster_seq'].value_counts()

s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5    392
s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4    392
s_cluster_8-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5    243
s_cluster_0-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4    152
s_cluster_2-s_cluster_3-s_cluster_0-s_cluster_3-s_cluster_0    140
                                                              ... 
s_cluster_7-s_cluster_3-s_cluster_0-s_cluster_3                  1
s_cluster_6-s_cluster_7-s_cluster_3-s_cluster_0-s_cluster_5      1
s_cluster_0-s_cluster_3-s_cluster_0-s_cluster_0                  1
s_cluster_2-s_cluster_8-s_cluster_5                              1
s_cluster_7-s_cluster_5-s_cluster_5-s_cluster_5                  1
Name: cluster_seq, Length: 305, dtype: int64

In [34]:
pred_dict[pred_dict.isna().any(axis=1)]['cluster_seq'].value_counts().head(20)

s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4    26
s_cluster_4-s_cluster_4-s_cluster_4                13
s_cluster_0-s_cluster_4-s_cluster_4-s_cluster_4    11
s_cluster_2-s_cluster_3-s_cluster_0                 7
s_cluster_7-s_cluster_4-s_cluster_4-s_cluster_4     6
s_cluster_4-s_cluster_4                             5
s_cluster_8-s_cluster_2-s_cluster_3-s_cluster_0     5
s_cluster_2-s_cluster_4-s_cluster_4-s_cluster_4     4
s_cluster_8-s_cluster_5-s_cluster_5-s_cluster_5     3
s_cluster_5-s_cluster_2-s_cluster_3-s_cluster_0     3
s_cluster_2-s_cluster_3                             3
s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5     3
s_cluster_3-s_cluster_1-s_cluster_2-s_cluster_3     3
s_cluster_5-s_cluster_5-s_cluster_5                 3
s_cluster_2-s_cluster_3-s_cluster_0-s_cluster_4     3
s_cluster_3-s_cluster_1                             3
s_cluster_7-s_cluster_1-s_cluster_2-s_cluster_3     3
s_cluster_1-s_cluster_5-s_cluster_5-s_cluster_5     3
s_cluster_1-s_cluster_2-s_cl

#### 概率树（中途剪枝版）

In [89]:
# 获取所有模型的函数，储存在models_dict内
def get_pred_group(iv_refs, dv_ref, dataset_name, threh): #iv_refs = ['college', 3, 's_cluster_5']; dv_ref = 's_cluster_3'
    os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\3-regression-model")
    # 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
    cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    models_dict = dict(zip([f'stage{i+1}' for i in range(1,dataset_info['stage_n'][dataset_name])],[0 for i in range(1,dataset_info['stage_n'][dataset_name])]))
    
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        ref_choice_list = iv_refs
        ref_dv = dv_ref
        choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]

        # 设定iv的ref
        for idx, choice in enumerate(ref_choice_list):
            var = choice_order_list[idx]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})

        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        # 储存模型
        models_dict['stage'+str(i+1)] = mlogit_res
    #获取预测
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_2.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        # input_X = df0.iloc[:,:-1]
        # input_y = df0['dv_stage']
        # 设置存储结果的df
        pred_cluster_info = df0[['prev_stage']].rename(columns={'prev_stage':'stage1'})
        pred_cluster_info['cluster_seq'] = pred_cluster_info['stage1']
        pred_cluster_info['prob'] = 0
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1)
        if i != 1:
            df0 = df0.loc[df0.index.intersection(threh_index)]
            df0['prev_stage']=pred_cluster_info['stage'+str(i)]
            print(threh_index)
        df0=df0.dropna()

        
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        for var in list(cat_dict.keys()):
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
            
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        pred_cluster_info['stage'+str(i+1)] = pd.Series(np.asarray(models_dict['stage'+str(i+1)].predict(train_X_sd)).argmax(1), index=train_X.index).apply(lambda x: 's_cluster_'+str(x))
        print(pred_cluster_info[pred_cluster_info['stage'+str(i+1)].isna()])
        # 剪枝
        stage_now = 'stage'+str(i+1)

        pred_cluster_info.loc[~pred_cluster_info[stage_now].isna(),'cluster_seq']=pred_cluster_info.dropna().apply(lambda x: x['cluster_seq']+'-'+x[stage_now] , axis=1)
        
        dict0 = pred_cluster_info.dropna()['cluster_seq'].value_counts(normalize=True).to_dict()
        pred_cluster_info.loc[~pred_cluster_info[stage_now].isna(),'prob']=pred_cluster_info.dropna().apply(lambda x: dict0[x['cluster_seq']], axis=1)
        
        #pred_cluster_info.loc[pred_cluster_info[stage_now]==np.nan]['prob']=1 #df.loc[]=xx并不改变df
       
        pred_cluster_info = pred_cluster_info[pred_cluster_info['prob']>=threh]
        threh_index = pred_cluster_info.index
        
    
    return pred_cluster_info

In [ ]:
pred_dict = get_pred_group(iv_refs = ['college', 3, 's_cluster_5'], dv_ref = 's_cluster_3', dataset_name='nlsy', threh=0.001)

In [100]:
pred_dict

,stage1,cluster_seq,prob,stage2,stage3,stage4,stage5,stage6,stage7,stage8
person_id,,,,,,,,,,
2,s_cluster_1,s_cluster_1-s_cluster_2-s_cluster_3-s_cluster_...,0.005000,s_cluster_2,s_cluster_3,s_cluster_0,s_cluster_0,s_cluster_0,s_cluster_0,s_cluster_3
6,s_cluster_0,s_cluster_0-s_cluster_8-s_cluster_5-s_cluster_...,0.017000,s_cluster_8,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5
16,s_cluster_5,s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_...,0.011500,s_cluster_5,s_cluster_5,s_cluster_2,s_cluster_3,s_cluster_0,s_cluster_2,s_cluster_3
17,s_cluster_5,s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_...,0.101500,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5
18,s_cluster_5,s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_...,0.101500,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5
...,...,...,...,...,...,...,...,...,...,...
12642,s_cluster_6,s_cluster_6-s_cluster_1-s_cluster_2-s_cluster_...,0.015933,s_cluster_1,s_cluster_2,s_cluster_3,s_cluster_0,s_cluster_2,s_cluster_3,NaN
12652,s_cluster_7,s_cluster_7-s_cluster_1-s_cluster_2-s_cluster_...,0.010381,s_cluster_1,s_cluster_2,s_cluster_3,s_cluster_0,NaN,NaN,NaN
12659,s_cluster_6,s_cluster_6-s_cluster_5-s_cluster_5-s_cluster_...,0.031867,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,s_cluster_5,NaN


In [105]:
df4 = pd.DataFrame(pred_dict.cluster_seq.value_counts())
df4[df4.cluster_seq>=10]

,cluster_seq
s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5,203
s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4,149
s_cluster_8-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5,137
s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4,132
s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4-s_cluster_4,84
...,...
s_cluster_6-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5,10
s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_2,10
s_cluster_0-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_5,10
s_cluster_2-s_cluster_3-s_cluster_0-s_cluster_2-s_cluster_3-s_cluster_0-s_cluster_0,10


In [96]:
pred_dict.loc[pred_dict['prob']>0.001]['stage1']=0
df3=pred_dict.iloc[:4,:4]
df3['k']=0
df3['stage1'][6]=np.nan
df3.loc[~df3.stage1.isna(),'k']=df3.dropna().apply(lambda x: x['prob']*2, axis=1)
df3

,stage1,cluster_seq,prob,stage2,k
person_id,,,,,
2,s_cluster_1,s_cluster_1-s_cluster_2-s_cluster_3-s_cluster_...,0.0050,s_cluster_2,0.010
6,NaN,s_cluster_0-s_cluster_8-s_cluster_5-s_cluster_...,0.0170,s_cluster_8,0.000
16,s_cluster_5,s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_...,0.0115,s_cluster_5,0.023
17,s_cluster_5,s_cluster_5-s_cluster_5-s_cluster_5-s_cluster_...,0.1015,s_cluster_5,0.203


#### 典型错误！

In [ ]:
### 大错特错的计算方法，stage之间并不独立，不能直接连乘！！！
# pred_dict = pred_dict.fillna(0)
# def combine_f(x):
#     if x[stage_now]==0:
#         return np.nan
#     else:
#         return x[stage_prev]+'-'+x[stage_now]
# def map_f(x):
#     if x[stage_now]==0:
#         return np.nan
#     else:
#         return dict0[x[stage_prev]+'-'+x[stage_now]]
# for i in range(1,8):#dataset_info['stage_n'][dataset_name]):
#     stage_prev = 'stage'+str(i)
#     stage_now = 'stage'+str(i+1)
#     dict0 = pred_dict[[stage_prev,stage_now]].apply(lambda x: combine_f(x) , axis=1).value_counts(normalize=True).to_dict()
#     pred_dict['prob'+str(i+1)]=pred_dict[[stage_prev,stage_now]].apply(lambda x: map_f(x), axis=1)
# pred_dict
# prob_info = pred_dict[['prob'+str(i) for i in range(2,9)]]
# threh = 0.0005
# df = prob_info
# # for i in range(1,7):
# #     df = df[df.iloc[:,:i].prod(axis=1)>0.01]
# df['prod'] = df.apply(lambda x: x.dropna().prod(), axis=1) 
# df[df['prod']>threh]

### individual

In [27]:
# 获取所有模型的函数，储存在models_dict内
def get_pred_ind(iv_refs, dv_ref, dataset_name, stage_k, person_id, event, weight): #iv_refs = ['college', 3, 's_cluster_5']; dv_ref = 's_cluster_3'
    # 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
    cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}

    # 各个datasets的基本信息，stage个数，cluster个数
    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    models_dict = dict(zip([f'stage{i+1}' for i in range(1,dataset_info['stage_n'][dataset_name])],[0 for i in range(1,dataset_info['stage_n'][dataset_name])]))
    
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        
        ref_choice_list = iv_refs
        ref_dv = dv_ref
        choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]

        # 设定iv的ref
        for idx, choice in enumerate(ref_choice_list):
            var = choice_order_list[idx]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})

        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        # 储存模型
        models_dict['stage'+str(i+1)] = mlogit_res
    # 获取预测
    for j in range(stage_k,dataset_info['stage_n'][dataset_name]):
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(j)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        #调整原数据
        input_X.loc[person_id][event]=1 
        input_X.loc[person_id]['event_n']=input_X.loc[person_id]['event_n']+1
        input_X.loc[person_id]['event_weighted_score']=input_X.loc[person_id]['event_weighted_score']+weight
        # entrophy, prev_cluster???
        input_y = df0['dv_stage']
        
        for var in list(cat_dict.keys()):
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        prob = models_dict['stage'+str(stage_k)].predict(train_X_sd).loc[person_id]
        prob.index=[f's_cluster_{i}' for i in range(dataset_info['cluster_n'][dataset_name])]
    
    return prob

In [29]:
pd.read_csv('psid/psid_stage_2.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()

,single_couple,couple_single,single_single,childbearing_no_yes,childbearing_yes_no,has_migration,education_level,first_job_income_quantile,num_of_event,event_weighted_score,event_entropy_score,prev_stage,gender,birth_year,race,dv_stage
person_id,,,,,,,,,,,,,,,,
4004,0,0,0,0,0,0,high-school,5,2,1.3,0.693147,s_cluster_5,female,1954,white,s_cluster_5
4007,0,0,1,0,0,0,high-school,4,3,2.1,1.098612,s_cluster_5,female,1960,white,s_cluster_8
4035,0,0,0,0,0,0,high-school,3,2,1.3,0.693147,s_cluster_0,male,1977,white,s_cluster_5
4172,0,0,0,1,0,0,college,3,3,1.8,1.098612,s_cluster_0,male,1949,white,s_cluster_0
4175,0,0,0,0,0,0,high-school,5,2,1.3,0.693147,s_cluster_8,male,1960,white,s_cluster_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9205001,0,0,0,0,0,0,high-school,5,2,1.3,0.693147,s_cluster_8,female,1968,other,s_cluster_8
9245001,0,0,0,0,0,0,high-school,5,2,1.3,0.693147,s_cluster_8,female,1960,other,s_cluster_8
9284171,0,0,0,0,0,0,high-school,5,2,1.3,0.693147,s_cluster_8,male,1962,other,s_cluster_8


In [28]:
get_pred_ind(iv_refs = ['college', 3, 's_cluster_5'], dv_ref = 's_cluster_3', dataset_name='psid', stage_k=2,person_id=4004, event='single_couple')

         Current function value: 1.487763
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.388028
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.326433
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.276594
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.279294
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.300871
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.181268
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.018218
         Iterations:

s_cluster_0    0.044300
s_cluster_1    0.017170
s_cluster_2    0.090324
s_cluster_3    0.143336
s_cluster_4    0.033165
s_cluster_5    0.323314
s_cluster_6    0.022288
s_cluster_7    0.074044
s_cluster_8    0.159713
s_cluster_9    0.092345
Name: 4004, dtype: float64

## 单次测试

In [150]:
input_X = df0.iloc[:,:-1]
input_y = df0["dv_stage"].replace({'dvcluster1':'a_dvcluster1'})
input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=ref_pv)
#input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=ref_firstjob)
input_X['education_level']=pd.Categorical(input_X['education_level'], categories=ref_edu)
cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
sc_X = StandardScaler()
train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

train_y = input_y
mlogit_mod = sm.MNLogit(train_y, train_X_sd)
mlogit_res = mlogit_mod.fit(method='bfgs')
parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues
    

         Current function value: 1.338097
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


In [50]:
k = parameters[0].apply(lambda x: math.exp(x))

k1 = k[['race_other','race_white']]
k1.index=[1,2]
k1

1    0.862887
2    1.369401
Name: 0, dtype: float64

In [78]:
dvcluster_list = [f'dvcluster{i}' for i in range(nlsw_cluster_n)]
coef_p_value_dict = dict(zip(dvcluster_list,[0 for i in range(nlsw_cluster_n)]))
for ref_dv in dvcluster_list:
    dv_rest_list = dvcluster_list
    dv_rest_list.remove(ref_dv)
    for i in range(parameters.shape[1]):
        l0=[ p_values[i], parameters[i], parameters[i].apply(lambda x: math.exp(x))]
        df = pd.concat(l0, axis=1)
        df.columns = ['p-value', 'coef', 'e_coef']
        coef_p_value_dict[ref_dv][dv_rest_list[i]+'_'+ref_edu+'_'+ref_firstjob+'_'+ref_pv] = df.loc[num_var_list+bool_var_list].to_dict('index')
    
        for idx, ch_cat in enumerate(cat_change_var_list):
            names = [i for i in list(df.index) if i.startswith(ch_cat)]
            df1 = df.loc[names]
            df1.index = [i.replace(ch_cat,ref_groups_iv[idx]) for i in names]
            coef_p_value_dict[ref_dv][dv_rest_list[i]+'_'+ref_edu+'_'+ref_firstjob+'_'+ref_pv][ch_cat] = df1.to_dict('index')

NameError: name 'ref_edu' is not defined

In [75]:
ss={}
ss[1][2]=3

KeyError: 1

In [ ]:
for i in list(models_dict.keys()):
    print(models_dict[i].llf)

In [ ]:
for i in range(2,8):
    df0 = pd.read_csv('nlsw_stage_'+str(i)+'.csv').drop('Unnamed: 0', axis=1).dropna()
    input_X = df0.iloc[:,:-1]
    input_y = df0["dv_stage"]
    ## 先处理control的cat var
    for var in list(control_dict.keys()):
        input_X[var] = pd.Categorical(input_X[var], categories=control_dict[var])
    ## 循环遍历所有ref搭配，并储存
    models_dict = {}
    for ref_pv in refs_dict['prev_stage']:
        input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=ref_pv)
        for ref_quan in refs_dict['first_job_income_quantile']:
            input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=ref_quan)
            for ref_edu in refs_dict['education_level']:
                input_X['education_level']=pd.Categorical(input_X['education_level'], categories=ref_edu)
                
                cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
                train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
                train_y = input_y
                mlogit_mod = sm.MNLogit(train_y, train_X, missing='raise')
                mlogit_res = mlogit_mod.fit(method='ncg')
                name = ref_pv[0]+'+'+str(ref_quan[0])+'+'+ref_edu[0]
                models_dict[name] = mlogit_res
    cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
    train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
    sc_X = StandardScaler()
    train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), columns=train_X.columns) 
    print(train_X_sd)
    train_y = input_y
    mlogit_mod = sm.MNLogit(train_y, train_X_sd)
    mlogit_res = mlogit_mod.fit(method='bfgs')

    y0 = mlogit_res.predict()
    ll =pd.DataFrame(train_y).dropna()
    ll.index = range(ll.shape[0])
    def findmax(x):
        mxx=x.max()
        y = list(x)
        return 's_cluster_'+str(y.index(mxx))

    k = pd.concat([ll,pd.DataFrame(y0).apply(lambda x: findmax(x),axis=1)],axis=1,ignore_index=True).apply(lambda x: x[0]==x[1],axis=1).value_counts(normalize=True)
    print(k[True])

In [71]:
train_y

0       s_cluster_4
1       s_cluster_0
2       s_cluster_0
4       s_cluster_2
5       s_cluster_6
           ...     
4232    s_cluster_7
4235    s_cluster_4
4239    s_cluster_5
4240    s_cluster_7
4241    s_cluster_2
Name: dv_stage, Length: 1759, dtype: object

In [142]:
y0 = mlogit_res.predict()
ll =pd.DataFrame(train_y).dropna()
ll.index = range(ll.shape[0])
def findmax(x):
    mxx=x.max()
    y = list(x)
    return 's_cluster_'+str(y.index(mxx))

pd.concat([ll,pd.DataFrame(y0).apply(lambda x: findmax(x),axis=1)],axis=1,ignore_index=True).apply(lambda x: x[0]==x[1],axis=1).value_counts(normalize=True)

In [112]:
pd.DataFrame(y0).apply(lambda x: findmax(x),axis=1)

0       s_cluster_0
1       s_cluster_2
2       s_cluster_6
3       s_cluster_2
4       s_cluster_6
           ...     
1946    s_cluster_1
1947    s_cluster_7
1948    s_cluster_1
1949    s_cluster_7
1950    s_cluster_7
Length: 1951, dtype: object

In [143]:
def findmax(x):
    mxx=x.max()
    y = list(x)
    return 's_cluster_'+str(y.index(mxx))

pd.concat([ll,pd.DataFrame(y0).apply(lambda x: findmax(x),axis=1)],axis=1,ignore_index=True).apply(lambda x: x[0]==x[1],axis=1).value_counts(normalize=True)

True     0.516373
False    0.483627
dtype: float64

In [55]:
pd.DataFrame(mlogit_res.predict())

,0,1,2,3,4,5,6,7,8
0,0.250169,0.050479,0.180217,0.070284,0.083352,0.046849,0.215806,0.057236,0.045608
1,0.068907,0.035347,0.331336,0.086530,0.090881,0.109337,0.206482,0.029130,0.042050
2,0.079694,0.028791,0.270353,0.083554,0.060833,0.135964,0.288296,0.023835,0.028681
3,0.049251,0.035031,0.366946,0.109909,0.039348,0.185082,0.139477,0.042498,0.032458
4,0.028691,0.017598,0.170932,0.064904,0.068805,0.007133,0.604101,0.004743,0.033093
...,...,...,...,...,...,...,...,...,...
1946,0.021706,0.669726,0.000134,0.020812,0.014778,0.037947,0.000165,0.172152,0.062579
1947,0.034801,0.290837,0.034652,0.039355,0.021112,0.094626,0.009043,0.366401,0.109172
1948,0.096811,0.238810,0.108151,0.079280,0.022202,0.090311,0.051077,0.197643,0.115716
1949,0.013979,0.128406,0.254042,0.000676,0.018335,0.228065,0.023920,0.305955,0.026625


In [149]:
for ref_pv in refs_dict['prev_stage']:
    print(ref_pv)

['s_cluster_0', 's_cluster_1', 's_cluster_2', 's_cluster_3', 's_cluster_4', 's_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8']
['s_cluster_1', 's_cluster_2', 's_cluster_3', 's_cluster_4', 's_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0']
['s_cluster_2', 's_cluster_3', 's_cluster_4', 's_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0', 's_cluster_1']
['s_cluster_3', 's_cluster_4', 's_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0', 's_cluster_1', 's_cluster_2']
['s_cluster_4', 's_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0', 's_cluster_1', 's_cluster_2', 's_cluster_3']
['s_cluster_5', 's_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0', 's_cluster_1', 's_cluster_2', 's_cluster_3', 's_cluster_4']
['s_cluster_6', 's_cluster_7', 's_cluster_8', 's_cluster_0', 's_cluster_1', 's_cluster_2', 's_cluster_3', 's_cluster_4', 's_cluster_5']
['s_cluster_7', 's_cluster_8', 's_cluster_0', 's

In [158]:
train_X.columns

Index(['num_of_event', 'birth_year', 'single_couple', 'couple_single',
       'single_single', 'childbearing_no_yes', 'childbearing_yes_no',
       'has_migration', 'education_level_high-school-noncompletion',
       'education_level_high-school', 'first_job_income_quantile_1',
       'first_job_income_quantile_2', 'first_job_income_quantile_3',
       'first_job_income_quantile_4', 'prev_stage_s_cluster_0',
       'prev_stage_s_cluster_1', 'prev_stage_s_cluster_2',
       'prev_stage_s_cluster_3', 'prev_stage_s_cluster_4',
       'prev_stage_s_cluster_5', 'prev_stage_s_cluster_6',
       'prev_stage_s_cluster_7', 'gender_female', 'race_black', 'race_other'],
      dtype='object')

In [176]:
## 先处理control的cat var
for var in list(control_dict.keys()):
    input_X[var] = pd.Categorical(input_X[var], categories=control_dict[var])
## 循环遍历所有ref搭配，并储存
models_dict = {}
for ref_pv in refs_dict['prev_stage']:
    input_X['prev_stage']=pd.Categorical(input_X['prev_stage'], categories=ref_pv)
    for ref_quan in refs_dict['first_job_income_quantile']:
        input_X['first_job_income_quantile']=pd.Categorical(input_X['first_job_income_quantile'], categories=ref_quan)
        for ref_edu in refs_dict['education_level']:
            input_X['education_level']=pd.Categorical(input_X['education_level'], categories=ref_edu)
            
            cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
            train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
            train_y = input_y
            mlogit_mod = sm.MNLogit(train_y, train_X, missing='raise')
            mlogit_res = mlogit_mod.fit(method='ncg')
            name = ref_pv[0]+'+'+str(ref_quan[0])+'+'+ref_edu[0]
            models_dict[name] = mlogit_res

Optimization terminated successfully.
         Current function value: 2.118301
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
Optimization terminated successfully.
         Current function value: 2.118301
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
Optimization terminated successfully.
         Current function value: 2.118301
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
Optimization terminated successfully.
         Current function value: 2.118301
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
Optimization terminated successfully.
         Current function value: 2.118301
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluation

In [41]:
results_as_html = models_dict['s_cluster_3+4+high-school'].summary().tables[1].as_html()

In [45]:
pd.read_html(results_as_html, header=0, index_col=0)[0][0]

KeyError: 0

In [99]:
# 定义multinomial fun
def mnlog_reg(cv_control_dict, input_data):
    #control_data = input_data[input_data['gender']==control_dict['gender'] & input_data['race']==control_dict['race'] & control_dict['year'][0]<=input_data['year']<=control_dict['year'][1]]
    # 定义储存dic
    num_var_list = ['num_of_event','event_weighted_score', 'event_entropy_score','birth_year']
    cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage','gender','race']
    control_var_list = ['gender','race','birth_year']
    bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration']
    dependent_var = ['dv_stage']
    nlsw_cat_dict.update(cv_control_dict)
    
    cycle_dict = {}
    for var in cat_var_list:
        l1=[]
        for cat in nlsw_cat_dict[var]:
            old_cats = nlsw_cat_dict
            old_cats.remove(cat)
            new_cats = [cat]+old_cats
            l1.append(new_cats)
        cycle_dict[var]=l1
    df_cat = control_data[cat_var_list]
    for idx, var in enumerate(cat_var_list):
        ref = reference_var[idx]
        l1 = list(input_data[var].unique())
        l1.remove(ref)
        l2 = [ref]+l1
        input_data[var] = pd.Categorical(X['categorical'], categories=l2)
    categorical_dummies = pd.get_dummies(input_data[cat_var_list], drop_first=True) #get dataframes
        
    # combine all the data
    df = input_data[num_var_list + bool_var_list]
    X_train = pd.concat([df, categorical_dummies], axis=1)    
    y_train = np.array(dependent_var)
    
    # fit mnlogistic model
    mlogit_mod = sm.MNLogit(y_train, X_train)
    mlogit_res = mlogit_mod.fit()
    
    parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
    p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues
    
    
    return mlogit_res
# parameters, std_error, loglikelihood,  p_values, t_values, X_train
# parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
#     p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues
    

In [ ]:
 # for each model, for each Reference Group in the dependent variable, pls output below dict:
{
 "single_couple": {"p-value": xx, "coef": xx, "e_coef": xx, "lrt": xx, "degrees_of_freedom": xx},  # 下面用"{...}"代替这个具体内容
 "couple_single": {...}, # 同上
 "single_single": {...}, # 同上
 "childbearing_no_yes": {...}, # 同上
 "childbearing_yes_no": {...}, # 同上
 "has_migration": {...}, # 同上
    # "central_nocentral" 这两个只出现在NLSY dataset中
    # "nocentral_central"
 "education_level": {
       "category0_category1": {...}, # category1/2/3: 是具体degree名称。下划线前面是reference group, 后面是target group
       "category0_category2": {...}, # if there are n categories, there should be (n-1)*(n-2) pairs
       "category0_category3": {...}, 
       "category1_category2": {...}, 
       "category1_category3": {...}, 
       "category2_category3": {...}, 
  }, 
 "first_job_income_quantile": {......}, # 同 education_level

 "num_of_event": {...}, # 同上
 "event_weighted_score": {...}, # 同上
 "event_entropy_score": {...}, # 同上

 "prev_stage": {......}, # 同 education_level
 "gender" # control variable不输出
 "birth_year" # control variable不输出
 "race" # control variable不输出
 "dv_stage" # DV不输出
}

In [69]:
list = [1,2,3]
list.insert(0,3)
list

[3, 1, 2, 3]

In [ ]:
# 生成假数据
X, y = make_classification(n_samples=3000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, random_state=1)
X = pd.DataFrame(X)
X.columns = ['numerical','boolean', 'categorical', 'ordinal']
X['boolean'] = pd.cut(X['boolean'], 2, labels=[0,1], ordered=False)
X['categorical']= pd.cut(X['categorical'], 5, labels=['a','b','c','d','e'], ordered=False) 
X['ordinal']= pd.cut(X['ordinal'], 10, labels=[f'level_{i}' for i in range(1,11)], ordered=False) 
map_ord1={'ordinal': [f'level_{i}' for i in range(1,11)]}

model = mnlog_reg(['numerical'],['boolean'],['categorical'],['b'],['ordinal'],map_ord1, X,y)

## 成功demo

In [10]:
# 生成假数据
X, y = make_classification(n_samples=3000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, random_state=1)
X = pd.DataFrame(X)
X.columns = ['numerical','boolean', 'categorical', 'ordinal']
X['boolean'] = pd.cut(X['boolean'], 2, labels=[0,1], ordered=False)
X['categorical']= pd.cut(X['categorical'], 5, labels=['level_2','level_1','level_3','level_4','level_5'], ordered=False) 
X['ordinal']= pd.cut(X['ordinal'], 10, labels=[i+1 for i in range(1,11)], ordered=False) 
#X['categorical'].cat.categories.remove('level_1')
X['categorical'] = pd.Categorical(X['categorical'], categories=['level_1','level_3','level_2','level_4','level_5'])

In [11]:
X1=pd.concat([X[['numerical']+['boolean']+['ordinal']], pd.get_dummies(X[['categorical']], drop_first=True)],axis=1)
y
mlogit_mod = sm.MNLogit(y, X1)
mlogit_res = mlogit_mod.fit()
mlogit_res.summary()

Optimization terminated successfully.
         Current function value: 0.932016
         Iterations 21


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                 3000
Model:                        MNLogit   Df Residuals:                     2986
Method:                           MLE   Df Model:                           12
Date:                Tue, 22 Feb 2022   Pseudo R-squ.:                  0.1516
Time:                        16:04:13   Log-Likelihood:                -2796.0
converged:                       True   LL-Null:                       -3295.8
Covariance Type:            nonrobust   LLR p-value:                2.319e-206
=======================================================================================
                y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
numerical              -0.4984      0.042    -11.756      0.000      -0.582      -0.415
boolean                -0.6921      0.107     -6.458      0.000      -0.902      -0.482
ordinal                -0.0646      0.018     -3.685      0.000      -0.099      -0.030
categorical_level_3     0.7695      0.127      6.076      0.000       0.521       1.018
categorical_level_2   -40.3076   1.86e+08  -2.16e-07      1.000   -3.65e+08    3.65e+08
categorical_level_4     2.8330      0.162     17.462      0.000       2.515       3.151
categorical_level_5     4.2148      0.733      5.747      0.000       2.777       5.652
---------------------------------------------------------------------------------------
                y=2       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
numerical               0.0196      0.037      0.525      0.600      -0.054       0.093
boolean                -0.4039      0.095     -4.252      0.000      -0.590      -0.218
ordinal                -0.0077      0.014     -0.539      0.590      -0.036       0.020
categorical_level_3     0.3340      0.102      3.285      0.001       0.135       0.533
categorical_level_2    -3.9212      1.013     -3.871      0.000      -5.907      -1.936
categorical_level_4     1.3321      0.151      8.800      0.000       1.035       1.629
categorical_level_5     3.3949      0.730      4.650      0.000       1.964       4.826
=======================================================================================
"""

In [ ]:
# 生成假数据
X, y = make_classification(n_samples=3000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, random_state=1)
X = pd.DataFrame(X)
X.columns = ['numerical','boolean', 'categorical', 'ordinal']
X['boolean'] = pd.cut(X['boolean'], 2, labels=[0,1], ordered=False)
X['categorical']= pd.cut(X['categorical'], 5, labels=['a','b','c','d','e'], ordered=False) 
X['ordinal']= pd.cut(X['ordinal'], 10, labels=[f'level_{i}' for i in range(1,11)], ordered=False) 

In [ ]:
categorical_list = ['categorical']
input_data = X
if len(categorical_list)==1:
    categorical_data = input_data[categorical_list]
else:
    categorical_data = np.array(input_data[categorical_list])    
categorical_dummies = OneHotEncoder(drop='first',
handle_unknown="error", sparse=False).fit(categorical_data)
categorical_dummies.transform(categorical_data)

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [ ]:
np.array(pd.concat([X[['numerical','boolean','ordinal']],pd.get_dummies(X[['categorical']], drop_first=True)], axis=1))

array([[0.664143224627229, '1', 6, ..., 1, 0, 0],
       [-0.25253161447316563, '0', 9, ..., 0, 0, 1],
       [0.9299551060061231, '0', 7, ..., 0, 0, 0],
       ...,
       [-1.5647936375030667, '0', 7, ..., 1, 0, 0],
       [4.153741239800722, '0', 6, ..., 1, 0, 0],
       [-0.5201549855271481, '1', 4, ..., 0, 1, 0]], dtype=object)

In [ ]:
ordinal_list = ['categorical', 'ordinal']
demanded_order = {
    'categorical':['a','b','c','d','e'],
    'ordinal': [f'level_{i}' for i in range(1,11)]
}
input_data=X
for var in ordinal_list:
    value = [i+1 for i in range(len(demanded_order[var]))] #demanded_order: ordinal变量想要的顺序的dict{var_name: demanded顺序list}
    key = demanded_order[var]
    map_ord = dict(zip(key, value))
    input_data[var] = input_data[var].replace(map_ord)
    x=pd.Categorical(input_data[var], [i+1 for i in range(len(demanded_order[var]))], ordered=True)

In [ ]:
x

[6, 9, 7, 5, 3, ..., 6, 5, 7, 6, 4]
Length: 3000
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

In [ ]:
value = [f'level_{i}' for i in range(1,6)]
key = ['a','ab','c','d','e']
map1 = dict(zip(key, value))
X['categorical'].replace(map1)

0       level_3
1       level_5
2       level_2
3       level_3
4       level_3
         ...   
2995    level_3
2996    level_3
2997    level_3
2998    level_3
2999    level_4
Name: categorical, Length: 3000, dtype: object

In [ ]:
    model_dict = {}
    # # ordinal：
    # for var in ordinal_list:
    #     value = [i+1 for i in range(len(demanded_order[var]))] #demanded_order: ordinal变量想要的顺序的dict{var_name: demanded顺序list}
    #     key = demanded_order[var]
    #     map_ord = dict(zip(key, value))
    #     input_data[var] = input_data[var].replace(map_ord)
    #     input_data[var] = pd.Categorical(input_data[var], [i+1 for i in range(len(demanded_order[var]))], ordered=True)
           
    # cat_var_list  
    # according to reference var